# Gappy

Gappy ("Gaps") is played on a square grid. The aim is to blacken some cells of a grid according to the following rules:

Each row and each column contains two black cells.

No black cells touch each other, not even diagonally.

Numbers outside the grid show the number of white cells between black cells in corresponding row or column.

- 逻辑游戏 Gappy （“Gops”）的玩法是根据以下规则将网格的某些单元格变黑：

1. 每行和每列包含两个黑色单元格。

2. 没有黑色单元格相互接触，甚至没有对角线接触。

3. 网格外的数字显示相应行或列中黑色单元格之间的白色单元格数。

The rules and examples can be found [here](https://www.janko.at/Raetsel/Gappy/index.htm):


In [2]:
def readGrid(path):
    with open(f"../assets/data/Gappy/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        for idx, g in enumerate(res):
            res[idx] = list(map(str, g))
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("10x10")
    print(grid)

    

[['1', '1', '3', '5', '1', '1', '3', '3', '3', '1'], ['2', '7', '1', '6', '1', '1', '2', '1', '1', '2']]


In [4]:
from ortools.sat.python import cp_model as cp

def GappySolver(m, n, grid):
    model = cp.CpModel()
    z = dict()
    x = dict()
    y = dict()
    # 存储决策变量的两个字典
    for i in range(m):
        for j in range(n):
            z[i, j] = model.NewBoolVar(name = f"z[{i},{j}]")
    for i in range(m):
        if grid[1][i] == '-':
            continue
        slot_length = int(grid[1][i]) + 2
        for j in range(n - slot_length + 1):
            x[i, j] = model.NewBoolVar(name = f"x[{i},{j}]")
    
    for j in range(n):
        if grid[0][j] == '-':
            continue
        slot_length = int(grid[0][j]) + 2
        for i in range(m - slot_length + 1):
            y[i, j] = model.NewBoolVar(name = f"y[{i},{j}]")
    
    # 添加约束
    for i in range(m - 1):
        for j in range(n - 1):
            model.Add(z[i, j] + z[i + 1, j] + z[i, j + 1] + z[i + 1, j + 1] <= 1)
    
    for j in range(n):
        if grid[0][j] == '-':
            continue
        
        slot_length = int(grid[0][j]) + 2
        for i in range(m - slot_length + 1):
            model.Add(z[i, j] >= 1 - (1 - y[i, j]))
            model.Add(z[i, j] <= 1 + (1 - y[i, j]))
            model.Add(z[i + slot_length - 1, j] >= 1 - (1 - y[i, j]))
            model.Add(z[i + slot_length - 1, j] <= 1 + (1 - y[i, j]))

    for i in range(m):
        if grid[1][i] == '-':
            continue
        
        slot_length = int(grid[1][i]) + 2
        for j in range(n - slot_length + 1):
            model.Add(z[i, j] >= 1 - (1 - x[i, j]))
            model.Add(z[i, j] <= 1 + (1 - x[i, j]))
            model.Add(z[i, j + slot_length - 1] >= 1 - (1 - x[i, j]))
            model.Add(z[i, j + slot_length - 1] <= 1 + (1 - x[i, j]))

    for i in range(m):
        model.Add(sum([z[i, j] for j in range(n)]) == 2 )
    for j in range(n):
        model.Add(sum([z[i, j] for i in range(m)]) == 2 )
    
    for i in range(m):
        if grid[1][i] == '-':
            continue
        
        slot_length = int(grid[1][i]) + 2
        model.Add(sum([x[i, j] for j in range(n - slot_length + 1)]) == 1)
    
    for j in range(n):
        if grid[0][j] == '-':
            continue
        slot_length = int(grid[0][j]) + 2
        model.Add(sum([y[i, j] for i in range(n - slot_length + 1)]) == 1)
        
    solver = cp.CpSolver()
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(m):
            for j in range(n):
                print(solver.Value(z[i, j]), end = " ")
            print()
        print()
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())

    else:
        print("Unable to find the OPTIMAL.")

if __name__ == "__main__":
    m, n, grid = readGrid("10x10")
    GappySolver(m, n, grid)


0 0 1 0 0 1 0 0 0 0 
1 0 0 0 0 0 0 0 1 0 
0 0 0 1 0 1 0 0 0 0 
1 0 0 0 0 0 0 1 0 0 
0 0 1 0 1 0 0 0 0 0 
0 0 0 0 0 0 1 0 1 0 
0 1 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 1 
0 1 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 1 

NumConflicts: 0
NumBranches: 0
WallTime: 0.014433000000000001
